In [1]:
from agentflow.env import VecGymEnv
from agentflow.agents import DDPG
from agentflow.buffers import BufferMap
from agentflow.state import NPrevFramesStateEnv
from agentflow.tensorflow.nn import dense_net
from agentflow.tensorflow.ops import normalize_ema
import tensorflow as tf

In [2]:
env_id = 'Pendulum-v0'
dqda_clipping = 1
clip_norm = False
hidden_dims = 32
hidden_layers = 1
activation = 'relu'

In [3]:
def build_policy_fn(hidden_dims,hidden_layers,output_dim,activation):
    def policy_fn(state,training=False):
        h = state
        #h = dense_net(h,hidden_dims,hidden_layers,activation=activation)
        BN = tf.layers.BatchNormalization()
        h = BN(h,training=training)
        with tf.control_dependencies(BN.updates):
            return tf.layers.dense(h,output_dim)
    return policy_fn

def build_q_fn(hidden_dims,hidden_layers,output_dim,activation):
    def q_fn(state,action,training=False):
        h = tf.concat([state,action],axis=1)
        #h = dense_net(h,hidden_dims,hidden_layers,activation=activation)
        BN = tf.layers.BatchNormalization()
        h = BN(h,training=training)
        with tf.control_dependencies(BN.updates):
            return tf.layers.dense(h,output_dim)
    return q_fn

In [4]:
env = VecGymEnv(env_id,n_envs=10)
env = NPrevFramesStateEnv(env,n_prev_frames=4,flatten=True)
state = env.reset()

state_shape = state.shape
action_shape = env.env.action_shape()
state_shape, action_shape

((10, 12), (10, 1))

In [5]:
policy_fn = build_policy_fn(hidden_dims,hidden_layers,1,activation)
q_fn = build_q_fn(hidden_dims,hidden_layers,1,activation)

In [6]:
tf.reset_default_graph()
agent = DDPG(state_shape[1:],action_shape[1:],policy_fn,q_fn,dqda_clipping,clip_norm)

W0928 21:05:43.668196 4519658944 deprecation_wrapper.py:119] From /Users/justinmaojones/rl/agentflow/agents/ddpg.py:40: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0928 21:05:43.688724 4519658944 deprecation_wrapper.py:119] From /Users/justinmaojones/rl/agentflow/agents/ddpg.py:55: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0928 21:05:43.773952 4519658944 deprecation.py:323] From <ipython-input-3-2f56b2c39105>:8: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0928 21:05:43.776058 4519658944 deprecation.py:506] From /Users/justinmaojones/rl/env/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer

In [7]:
agent.outputs

{'loss': <tf.Tensor 'add:0' shape=<unknown> dtype=float32>,
 'loss_Q': <tf.Tensor 'TDLearning/mul_1:0' shape=<unknown> dtype=float32>,
 'loss_policy': <tf.Tensor 'DpgLearning/mul:0' shape=(?,) dtype=float32>,
 'policy_train': <tf.Tensor 'policy/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 'policy_eval': <tf.Tensor 'policy_1/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 'Q_action_train': <tf.Tensor 'Q/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 'Q_policy_train': <tf.Tensor 'Q_1/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 'policy_ema': <tf.Tensor 'policy_2/dense/BiasAdd:0' shape=(?, 1) dtype=float32>,
 'Q_ema_state2': <tf.Tensor 'Q_2/dense/BiasAdd:0' shape=(?, 1) dtype=float32>}

In [7]:
#agent = DDPG(state_shape,action_shape,policy_fn,q_fn,dqda_clipping,clip_norm)
replay_buffer = BufferMap()

"""
obs: n_env x [obs_shape]
states: n_env x [state_shape]
"""
action = agent.act(sess,state)
ob, reward, done, info = env.step(action)
replay_buffer.append({'state':state,'action':action,'reward':reward,'done':done})

ValueError: Variable policy/batch_normalization/gamma already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-3-2f56b2c39105>", line 6, in policy_fn
    h = BN(h,training=training)
  File "/Users/justinmaojones/rl/agentflow/agents/ddpg.py", line 56, in build_model
    policy_train = self.policy_fn(inputs['state'],training=True)
  File "/Users/justinmaojones/rl/agentflow/agents/ddpg.py", line 34, in __init__
    self.build_model()
  File "<ipython-input-6-95371d1f0567>", line 2, in <module>
    agent = DDPG(state_shape[1:],action_shape[1:],policy_fn,q_fn,dqda_clipping,clip_norm)
  File "/Users/justinmaojones/rl/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
